# James Jones
## 12-07-2022

In [2]:
# Import Libraries
import os, time, json
import tmdbsimple as tmdb 
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm_notebook

In [3]:
# Create new folder for tmdb data (as to not overwrite the previous 2000, 2001 data)
FOLDER = "Data_Part4/"
os.makedirs(FOLDER, exist_ok = True)
os.listdir(FOLDER)

['tmdb_api_results_2010.json',
 ' final_tmdb_data_2018.csv.gz',
 ' final_tmdb_data_2014.csv.gz',
 ' final_tmdb_data_2016.csv.gz',
 'tmdb_api_results_2011.json',
 'tmdb_api_results_2016.json',
 ' final_tmdb_data_2012.csv.gz',
 ' final_tmdb_data_2010.csv.gz',
 'tmdb_api_results_2017.json',
 'tmdb_api_results_2018.json',
 ' final_tmdb_data_2019.csv.gz',
 ' final_tmdb_data_2015.csv.gz',
 'tmdb_api_results_2014.json',
 'tmdb_api_results_2015.json',
 ' final_tmdb_data_2017.csv.gz',
 'final_tmdb_data_2009.csv.gz',
 'tmdb_api_results_2019.json',
 '.ipynb_checkpoints',
 ' final_tmdb_data_2013.csv.gz',
 'tmdb_api_results_2012.json',
 'tmdb_api_results_2009.json',
 'tmdb_api_results_2013.json',
 ' final_tmdb_data_2011.csv.gz']

In [4]:
# Load in TMDB API cedentials
with open('/Users/jamesjones/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [5]:
# Assign API key
tmdb.API_KEY = login['api-key']

In [6]:
# Define a our function to get movie with the rating (certification) included
imdb_id = tmdb.Movies('imdb_id')

def get_movie_with_rating(imdb_id):
    # Get the movie object for the current id
    movie = tmdb.Movies(imdb_id)

# Save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()

# Loop through countries in releases
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
        # Save a "certificatio" key in the info dictionary 
            info['certification'] = c['certification']
    return info

# This function was originally built in "Project 3, Part 2"

In [7]:
# Define function to write new .json files

def write_json(new_data, filename): 
    # Appends a list of records (new_data) to a json file (filename)
     #Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/
    
    with open(filename, 'r+') as file:
        
        # First we load existing data into a dict.
        file_data = json.load(file)
        
        # Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
                
        # Sets file's current position at offset.
        file.seek(0)
        
        # Convert back to json.
        json.dump(file_data, file)

# This function was originally built in "Project 3, Part 2"

### Do to time, and processing restraints, we will retrieve movies from 2009-2019.
- Do to the nature of the pandemic, 2020-2022 likely cause largely different trends in cinema
  - Therefore, we will use a likely more stable time frame from pre-pandemic years

In [8]:
YEARS_TO_GET = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 
                2018, 2019]

In [9]:
# Load in Title Basics data created in Part 1
basics = pd.read_csv('Data/title_basics.csv.gz')
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,126,Drama


In [9]:
# Create a for loop to filter (loop) through years
    # OUTER LOOP
for YEARS in tqdm_notebook(YEARS_TO_GET, desc = 'YEARS', position = 0):

# Define the JSON file to store results for year
    # This will show at the beginng 
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEARS}.json'
    
    # Check to see if our file exists
    file_exists = os.path.isfile(JSON_FILE)
    
    # Now, create the file if it exists, or leave it if it does exist
    if file_exists == False:
    
        # Save an empty dict to a new json file to append later
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb_id':0}],f)
        
    # Now, save our specific years as the current df YEAR = 2000
    df_2009 = basics.loc[basics['startYear'] == YEARS].copy()
    df_2009.head(3)

    # Save movie ids from the year 2000 to a list
    movie_ids = df_2009['tconst'].copy()
    movie_ids

    # Check for any previous data
    previous_df = pd.read_json(JSON_FILE)
    previous_df # Will be empty at first
    
    # Filter out any ids that already exist in our JSON_FILE (helps when returning to our problem later)
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
# Get index and movie id from list
    # INNER loop

    for movie_id in tqdm_notebook(movie_ids_to_get,
                                      desc = f'Movies from {YEARS}',
                                      position = 1,
                                      leave = True):
            # Make attempt to retrieve data for movie id
        try:
            temp = get_movie_with_rating(movie_id) # Our first function

                # Extend results of our file with our other function
            write_json(temp, JSON_FILE)

                # Create a short sleep (helps bog down the server less)
            time.sleep(0.02)

            # If it fails, make a dict with just the id
        except Exception as e:
            continue
            
        # Finally, store our info in the JSON_FILE
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER} final_tmdb_data_{YEARS}.csv.gz",
                            compression = 'gzip', index = False)

YEARS:   0%|          | 0/11 [00:00<?, ?it/s]

Movies from 2009:   0%|          | 0/3371 [00:00<?, ?it/s]

Movies from 2010:   0%|          | 0/3788 [00:00<?, ?it/s]

Movies from 2011:   0%|          | 0/4162 [00:00<?, ?it/s]

Movies from 2012:   0%|          | 0/4443 [00:00<?, ?it/s]

...

Movies from 2015:   0%|          | 0/4938 [00:00<?, ?it/s]

Movies from 2016:   0%|          | 0/5156 [00:00<?, ?it/s]

Movies from 2017:   0%|          | 0/5530 [00:00<?, ?it/s]

Movies from 2018:   0%|          | 0/5650 [00:00<?, ?it/s]

Movies from 2019:   0%|          | 0/5751 [00:00<?, ?it/s]